In [ ]:
import magpylib as magpy
import numpy
import pandas
import matplotlib.pyplot as plt


: 

In [ ]:
Br_max = 1210 #mT
d = (3/16)*25.4
h = (1/8)*25.4

: 

In [ ]:
M = Br_max


: 

In [ ]:
magnet = magpy.magnet.Cylinder(magnetization = (0, 0, M), dimension=(d, h), position=(0, 0, 33))
sensor = magpy.Sensor(position=(-5, 0, 0))
B = magnet.getB(sensor)
print(B)
sensor.move([(0.5, 0, 0), (0, -0.5, 0)])
print(sensor.position)

: 

In [ ]:
sensor

: 